# Credit approvals


Author: Nick Van Bergen <br><br>
**To view fully rendered notebook click [here](https://nbviewer.org/github/nvbergen/DC_project_CC_apps/blob/566ba446ee359173d6cccd8b81a8f2b942c5a742/Code/Credit%20approval%20Project.ipynb#top)**

This project inspired by guided project from [datacamp](https://app.datacamp.com/learn/projects/558). <br>
The data souce is from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/credit+approval) <br> 





<a id = "top"></a>

---
## Contents
1. [Background and problem statement](#background)
1. [Library Imports](#imports)
1. [Data Ingestion](#ingest)
    * Data Description
    * Data set dictionary Column Names
1. [Data set Inspection](#inspect)
1. [Date Cleaning](#cleaning)
1. [EDA](#eda)
    * Location
    * Variability
    * Distribution
    * Correlation
1. Data Preprocessing
1. Logistic Regression Classifier
    1.  Fit
    1. score 
    1. grid search
1. [Conclusions and Recommendations](#conclusions)

<a id = "background"></a>

---
## Background and problem statement.
[[Back to contents]](#top) *-* [[Next Section: Library Imports]](#imports)<br> <br>
There was a time when credit card applications were conducted in-person or over the phone. Data was collected by a human and then entered into the issuer's system for approval. The goal of a credit card issuer is to well _issue credit_. The issuer will earn a profit on the interest against the money borrowed. There are several risks to the issuer. One of these risks is issuing credit to a risky borrower whom fails in their contractual obligation to repay the lender. <br><br>
Banks beleive that there are objective factors that can predict if a borrower is apt to fail in their obligation and thus influence an issuers decision to approve or decline a prospective borrower from a credit card. <br><br>
There are operational risks associated with collecting and inputing data as well as decisioning the application if left totally up to a human operator. Thankfully, we can utilize some statistical learning methods on past applications to speed up the approval process. It is believed that most, if not all, credit issuers today utilize some form of **machine learning** to mitigate their business and operational risks. <br><br>
**Problem Statement:** _Can machine learning provide us with an accurate solution to predict if a borrower will be approved fro credit cards or not?_ 

<a id = "imports"></a>

---
## Imports
[[Back to contents]](#top) *-* [[Next Section: Data Ingestion]](#ingest)<br> <br>

In [2]:
#analytical and visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#modeling tools
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

#model
from sklearn.linear_model import LogisticRegression


#model evaluations
from sklearn.metrics import confusion_matrix

<a id="ingest"></a>

---
## Data Ingestion
[[Back to contents]](#top) *-* [[Next Section: Data Inspection]](#inspect)<br> <br>

In [12]:
apps = pd.read_csv("../data/crx.data", header = None)

In [13]:
apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


### Data Descriptions

The data set has been **anonymized** to protect sensitive information under general view of public analysts. <br> 
> This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. <br>

### Column Names
The website does offer an appropriate description of the columns. However, per above, there is no way to build an intuition from the information at hand. 

>Attribute Information:<br>
A0: b, a. <br>
A1: continuous. <br>
A2: continuous. <br>
A3: u, y, l, t. <br>
A4: g, p, gg. <br>
A5: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff. <br>
A6: v, h, bb, j, n, z, dd, ff, o. <br>
A7: continuous. <br>
A8: t, f. <br>
A9: t, f. <br>
A10: continuous. <br>
A11: t, f. <br>
A12: g, p, s. <br>
A13: continuous. <br>
A14: continuous. <br>
A15: +,- (class attribute)<br>

[Source](https://archive.ics.uci.edu/ml/datasets/credit+approval) <br>
<br>
There are many notebooks that explain _probable_ feature names. One such blog describes the features, in order, as:
>Probable Feature Names: <br> 
A0: `Gender` <br>
A1: `Age` <br>
A2: `Debt` <br>
A3: `Married` <br>
A4: `BankCustomer` <br>
A5: `EducationLevel` <br>
A6: `Ethnicity` <br>
A7: `YearsEmployed` <br>
A8: `PriorDefault` <br>
A9: `Employed` <br>
A10: `CreditScore` <br>
A11: `DriversLicense` <br>
A12: `Citizen` <br>
A13: `ZipCode` <br>
A14: `Income` <br>
A15: `ApprovalStatus.` <br> 

[Source](http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html)

### Data Dictionary and column headings

Using the above information we will develop a full data dictionary below and map to our dataframe:

| Index | Feature name*    | Type                  | Value Range                                                             |                     Notes                     |
|:-----:|------------------|-----------------------|-------------------------------------------------------------------------|:---------------------------------------------:|
| 0     | _Gender_         | Categorical           | `b`, `a`                                                                | Definitions unknown                           |
| 1     | _Age_            | Numeric, Continuous   |                                                                         | Decimalized, in years                         |
| 2     | _Debt_           | Numeric, Continuous   |                                                                         | Definitions unknown                           |
| 3     | _Married_        | Categorical           | `u`, `y`, `l`, `t`                                                      | Definitions unknown                           |
| 4     | _BankCustomer_   | Categorical           | `g`, `p`, `gg`                                                          | Definitions unknown                           |
| 5     | _EducationLevel_ | Categorical           | `c`, `d`, `cc`, `i`, `j`, `k`, `m`, `r`, `q`, `w`, `x`, `e`, `aa`, `ff` | Definitions unknown                           |
| 6     | _Ethnicity_      | Categorical           | `v`, `h`, `bb`, `j`, `n`, `z`, `dd`, `ff`, `o`                          | Definitions unknown                           |
| 7     | _YearsEmployed_  | Numeric, Continuous   |                                                                         | Decimalized, in years                         |
| 8     | _PriorDefault_   | Categorical           | `t`, `f`                                                                | True = `t` = 1 <br> False = `f` = 0            |
| 9     | _Employed_       | Categorical           | `t`, `f`                                                                | True =  `t` = 1 <br> False = `f` = 0            |
| 10    | _CreditScore_    | Numeric, Continuous   |                                                                         |                                               |
| 11    | _DriversLicense_ | Categorical           | `t`,`f`                                                                 | True =  `t` = 1 <br> False = `f` = 0            |
| 12    | _Citizen_        | Categorical           | `g`, `p`, `s`                                                           | Unknown Definition                            |
| 13    | _ZipCode_        | Numeric, **discrete** |                                                                         | This should be **Categorical** and non-numeric                                              |
| 14    | _Income_         | Numeric, Continuous   |                                                                         |                                               |
| 15    | _ApprovalStatus_ | Categorical           | `+`,`-`                                                                 | `+` = Approved = 1 <br>  `-` = Not Approved = 0 |

In [19]:
#apply new column names to dataframe
apps.columns = ["Gender", "Age", "Debt", 
"Married", "BankCustomer", "EducationLevel", 
"Ethnicity", "YearsEmployed", "PriorDefault", 
"Employed", "CreditScore", "DriversLicense", 
"Citizen", "ZipCode", "Income", "ApprovalStatus"]
apps.head(3)

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.50,u,g,q,h,1.50,t,f,0,f,g,00280,824,+


<a id="inspect"></a>

---
## Data Inspection
[[Back to contents]](#top) *-* [[Next Section]](#next)<br> <br>
In this section, I would like to inspect the dataset overal to match the dictiony to our data. 


In [20]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         690 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [23]:
apps.describe()

,Debt,YearsEmployed,CreditScore,Income
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


I am supposed to have 6 numeric variables, and the `df.describe()` method only returned four. <br><br>
The `df.describe()` method is useful because, by default it will take the _numeric_ columns automatically. After applying this method, we see that only _Debt_, _YearsEmployed_, _CreditScore_, and _Income_ were returned. <br><br>
I am missing _Age_ and _ZipCode_, which were not returned due to the datatype being `object` as opposed to `int` or `float`.

In [28]:
#using .difference() we can exclude normally numeric columns and use .describe() on our categorical features. 
apps[apps.columns.difference(list(apps.describe().columns))].describe()

,Age,ApprovalStatus,BankCustomer,Citizen,DriversLicense,EducationLevel,Employed,Ethnicity,Gender,Married,PriorDefault,ZipCode
count,690,690,690,690,690,690,690,690,690,690,690,690
unique,350,2,4,3,2,15,2,10,3,4,2,171
top,?,-,g,g,f,c,f,v,b,u,t,00000
freq,12,383,519,625,374,137,395,399,468,519,361,132


The above process shows we got lucky with _Age_ column showing that there are 12 values that take the `?` form. <br><br>
Aslo, of note in the _ZipCode_ column is the value `00000` which indicates a missing value and the fact that this column has taken the datatype `object`. <br>Is this datatyping **incorrect** though? At least for _ZipCode_ it seems appropriate to not attempt to describe this data numerically since the number only represents a physical location and not a measurement. <br><br>
We will still need to find and deal with missing values in our dataset before we can explore the data in more detail. 

<a id="cleaning"></a>

---
## Data Cleaning
[[Back to contents]](#top) *-* [[Next Section: EDA]](#EDA)<br> <br>
We are hunting down missing values and addressing them by ascribing the `np.nan()` type to the datum that is missing.<br><br> 
We can do this rather easily, however, there are some other things that should be checked, or verified in this step. <br>
<br>
The goals of this section are to identify values that are truly missing and to ensure that datatypes make sense given our codebook. 


### First column investigation: _Gender_
The data dictionary shows that there should only be two values `b` or `a` in our first column _Gender_.

In [33]:
apps['Gender'].describe()

count     690
unique      3
top         b
freq      468
Name: Gender, dtype: object

Seeing that we have 3 values instead of 2: lets see what they are.

In [34]:
apps['Gender'].unique()

array(['b', 'a', '?'], dtype=object)

I will replace obvious missing values with numpy's `NaN` data type in the _Gender_ column.

In [36]:
apps['Gender'] = apps['Gender'].replace('?', np.nan)

In [39]:
apps['Gender'].isna().sum()

12

Going forward, in each column, replacing `?` with `np.nan` using the same method above afterwhich we can see the total number of missing values and their proportion to the overall dataset.

In [42]:
apps = apps.replace('?', np.nan)

In [62]:
print(apps.isna().sum())
print('total number NaN values is ', apps.isna().sum().sum())
print("Proportion of NaN to data set is ", apps.isna().sum().sum() / len(apps['Debt']))

Gender            12
Age               12
Debt               0
Married            6
BankCustomer       6
EducationLevel     9
Ethnicity          9
YearsEmployed      0
PriorDefault       0
Employed           0
CreditScore        0
DriversLicense     0
Citizen            0
ZipCode           13
Income             0
ApprovalStatus     0
dtype: int64
total number NaN values is  67
Proportion of NaN to data set is  0.09710144927536232


### Remaining columns
I will go through each column to attempt to understand the ranges of data and see if we can identify any other obvious candidates to be `missing` values. 

#### Column: _Age_

In [81]:
apps['Age'].describe()

count       678
unique      349
top       22.67
freq          9
Name: Age, dtype: object

The `apps['Age'].describe()` method indicates this is an object column when it should be numeric. I will cast these values as float64's. 

In [82]:
apps['Age'] = apps['Age'].astype(float)
apps['Age'].describe()

count    678.000000
mean      31.568171
std       11.957862
min       13.750000
25%       22.602500
50%       28.460000
75%       38.230000
max       80.250000
Name: Age, dtype: float64

Curiously, the minimum age in the dataset is rather young at 13.75 years, assuming, of course, that our assumed label is age in years. Further investigation should be done in the EDA section. 

#### Columns: _Debt_
This should also be a continuous numeric variable. Let's check it. 

In [83]:
apps['Debt'].describe()

count    690.000000
mean       4.758725
std        4.978163
min        0.000000
25%        1.000000
50%        2.750000
75%        7.207500
max       28.000000
Name: Debt, dtype: float64

This tells us taht we have no empty values and it is a continuous variable. What we do not know, however, are the units of these values. Does the _median_ value of `2.75` mean 27,500 dollars, 2.75 dollars, 2.75 million? We just don't know so we will ignore explanatory information for now. <br><br>
This column appears ok as it is right now in terms of reasonableness. 

#### Column: _Married_
This column can take on the following values `u`, `y`, `l`, `t` of which we do not have further definition. I will check them for reasonableness by looking at the unique values in the series.  

In [84]:
apps['Married'].unique()

array(['u', 'y', nan, 'l'], dtype=object)

We have some empty values in the dataset, and values reported within are within our available value set `u`, `y` and `l` are all available values in this column. 

In [128]:
apps['Married'].value_counts()

u    519
y    163
l      2
Name: Married, dtype: int64

#### Column: _Bank Customer_
This is a categorical column that can take on the values of `g`, `p`, and `gg`. As before, we do not know the definitions of these values as they have been previously anonymized. 

In [129]:
print(apps['BankCustomer'].unique())
print(apps['BankCustomer'].value_counts())

['g' 'p' nan 'gg']
g     519
p     163
gg      2
Name: BankCustomer, dtype: int64


#### Column: _Education Level_
Values in this categorical variable's column can take the values `c`, `d`, `cc`, `i`, `j`, `k`, `m`, `r`, `q`, `w`, `x`, `e`, `aa`, and `ff`. We do not know the definitions of these values either. We are simply going to check for reasonableness against what we know already. 

In [131]:
print(apps['EducationLevel'].unique())
print(apps['EducationLevel'].value_counts())

['w' 'q' 'm' 'r' 'cc' 'k' 'c' 'd' 'x' 'i' 'e' 'aa' 'ff' 'j' nan]
c     137
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
r       3
Name: EducationLevel, dtype: int64


Eyeballing these values may be a bit difficult to do. An easy way is to measure the `len()` of resulting array and see if that matches what we expect.

In [94]:
len(apps['EducationLevel'].unique())

15

I expected 14 categories, which the data had + a `NaN` type. 

#### Column: _Ethnicity_
This column can take the values `v`, `h`, `bb`, `j`, `n`, `z`, `dd`, `ff`, `o` of which we have no operating definitions. Again we are only checking for reasonableness and to clean the data if needed. 

In [134]:
print(apps['Ethnicity'].unique())
print(apps['Ethnicity'].value_counts())
print(len(apps['Ethnicity'].unique()))

['v' 'h' 'bb' 'ff' 'j' 'z' nan 'o' 'dd' 'n']
v     399
h     138
bb     59
ff     57
j       8
z       8
dd      6
n       4
o       2
Name: Ethnicity, dtype: int64
10


We expected 9 and were returned 10 on account of the `NaN` value. 

#### Column: _Years Employed_
We can guess at this given our column label (also a guess). Should be a continuous numeric variable. We can see that there are no empty `NaN` values and we have a complete series. Assuming the label is correct, what is interesting is where our 75th percentile is; `2.625` years. Borrowers in this dataset were mostly newer employees at their given firms. We do not know the nature of these employers or if all of our datapoints were from some group that would only average less than 5 years at a company before applying for credit. 

In [97]:
apps['YearsEmployed'].describe()

count    690.000000
mean       2.223406
std        3.346513
min        0.000000
25%        0.165000
50%        1.000000
75%        2.625000
max       28.500000
Name: YearsEmployed, dtype: float64

#### Column: _Prior Default_
This coulumn shoud be `t` for _True_ (or 1) or `f` for _False_ (or 0)

In [135]:
print(apps['PriorDefault'].describe())
print(apps['PriorDefault'].value_counts())

count     690
unique      2
top         t
freq      361
Name: PriorDefault, dtype: object
t    361
f    329
Name: PriorDefault, dtype: int64


In [136]:
apps['PriorDefault'].value_counts(normalize = True)

t    0.523188
f    0.476812
Name: PriorDefault, dtype: float64

This would seem to have a heavy influence on credit card approvals. Again, our guess is that this column is representing prior defaults. No data cleaning was done in this step. 
<br><br>
There is an added benefit here of seeing that our data, in this category, is nearly balanced.

#### Column: _Employed_
This column label describes if our borrower is employed at the time of application and can take only one of two values, `t` or `f`. 

In [103]:
apps['Employed'].describe()

count     690
unique      2
top         f
freq      395
Name: Employed, dtype: object

In [139]:
print(apps['Employed'].unique())
print(apps['Employed'].value_counts())
print(apps['Employed'].value_counts(normalize= True))

['t' 'f']
f    395
t    295
Name: Employed, dtype: int64
f    0.572464
t    0.427536
Name: Employed, dtype: float64


#### Column: _Credit Score_
This column should be a continuous numeric column. Let's look at `.describe()` to see if the values make sense. 

In [105]:
apps['CreditScore'].describe()

count    690.00000
mean       2.40000
std        4.86294
min        0.00000
25%        0.00000
50%        0.00000
75%        3.00000
max       67.00000
Name: CreditScore, dtype: float64

Well, without knowing the unit of measurement, we can't really say if this makes sense or not right off the bat. What we can remark on is the extreme maximum value of `67.00`. Not to worry, however, we will confirm outliers in the next section and then remove them at that time. 

#### Column: _Drivers License_
This column should only take one of two truth values, `t` or `f`. Having a license may indicate responsibility. But, who knows, we are only guessing at the labels and are not sure of the veracity of _"Having a licence is an indicator of financial responsibility."_ Let's continue and look for reasonableness here. 

In [127]:
apps['DriversLicense'].value_counts()

f    374
t    316
Name: DriversLicense, dtype: int64

The existing data fits our working description. No cleaning or transformations done in this step. 

#### Column: _Citizen_
This column should take one of three values, if available, `g`, `p`, and `s`. Like our other columns, we do not have working definitions of these values or any intuition on what they mean. If any are offered, they are pure speculation. 

In [107]:
apps['Citizen'].unique()

array(['g', 's', 'p'], dtype=object)

And we can check a short description of such.

In [110]:
apps['Citizen'].describe()

count     690
unique      3
top         g
freq      625
Name: Citizen, dtype: object

#### Column: _Zip Code_
Now we come to Zip Code. The code book and data dictionary states that this is a numeric variable which is continuous. However, our best guess at a label indicates that this should be zip code as it is 5 digits in length. If we assume that this is a zip code, then we should **not** treat these values as numeric variables. 

In [117]:
print(apps['ZipCode'].describe())

count       677
unique      170
top       00000
freq        132
Name: ZipCode, dtype: object


In [118]:
apps[apps['ZipCode'] == '00000'].describe()

,Age,Debt,YearsEmployed,CreditScore,Income
count,129.000000,132.000000,132.000000,132.000000,132.000000
mean,38.581395,7.396742,3.705076,4.250000,1161.962121
std,15.535516,6.218740,4.928837,6.179719,2515.489060
min,15.830000,0.000000,0.000000,0.000000,0.000000
25%,24.080000,1.916250,0.165000,0.000000,0.000000
50%,36.080000,6.270000,1.542500,1.000000,47.000000
75%,49.830000,11.062500,5.406250,7.250000,1000.000000
max,80.250000,28.000000,28.500000,40.000000,15000.000000


This data field presents more of a challenge for us for analysis. <br> <br> Here is what we know:
* There are 677 non null data points. 
* There are 170 unique entries
* The most frequent data value is `'00000'`
* If we consider `'00000'` as a missing value, and then discard those observations, we would lose 132 rows of data.

A possible solution would be to drop the column entirely. Something we will investigate during [EDA](#eda)

#### Column: _Income_
This is another seemingly relevent column that could influence an algorithm's performance. However, there is no way to form an intuition about this data since we do not know for sure where it comes from. All we are guessing is the label and looking for anything that might pop out to us.  

In [119]:
apps['Income'].describe()

count       690.000000
mean       1017.385507
std        5210.102598
min           0.000000
25%           0.000000
50%           5.000000
75%         395.500000
max      100000.000000
Name: Income, dtype: float64

Already we can see that without knowing units we might have to deal with outliers. A max value of `100,000` doesn't really tell us much other than it is very far from the 75th percentile. 

In [121]:
apps[apps['Income']==100000].describe()

,Age,Debt,YearsEmployed,CreditScore,Income
count,1.0,1.0,1.0,1.0,1.0
mean,17.5,22.0,0.0,0.0,100000.0
std,NaN,NaN,NaN,NaN,NaN
min,17.5,22.0,0.0,0.0,100000.0
25%,17.5,22.0,0.0,0.0,100000.0
50%,17.5,22.0,0.0,0.0,100000.0
75%,17.5,22.0,0.0,0.0,100000.0
max,17.5,22.0,0.0,0.0,100000.0


We only have one value at the maximum level of `100,000`. This will be addressed in a later section handling outliers in EDA.

#### Column: _Approval Status_
Our final column is our most important. This column is, for sure, our `class attribute` label. This is the value that our machine learning algorithm will want to predict with a high degree of accuracy and recall. <br><br>
I will apply a function to this column to _binarize_ it since we know we will need to keep it. That is, I am going to take the string values and convert them to numeric into a single column. 

In [124]:
apps['ApprovalStatus'] = apps['ApprovalStatus'].apply(lambda x: 1 if x == '+' else 0)

In [126]:
apps['ApprovalStatus'].value_counts()

0    383
1    307
Name: ApprovalStatus, dtype: int64

<a id= 'eda'></a>

---
## Exploratory Data Analysis
[[Back to contents]](#top) *-* [[Next Section: Data Preprocessing]](#preprocess)<br> <br>
This section will look at 4 kinds of analysis on our data set:
1. Location Estimates
2. Data Variability
3. Data Distribution
4. Correlation

## Data Preprocessing

## Machine Learning Model

### Fit Model

### Score Evaluate Model 

### Grid Search

<a id="conclusions"></a>
## Conclusions and Recommendations
---
[[Back to contents]](#top)

Here we discuss the general conclusions made. 

Conclusions!
</a>